# Adversity Outcome Prediction Training Pipeline
## Aim
To implement an adversity outcome prediction training pipeline to benchmark the metrics of logistic regression models applied to the authentic and synthetic MIMIC-III data processed through the FIDDLE pipeline outlined in Tang et al. 2020. 


# Differences between FIDDLE and Yoon et al. 2023 features for mortality prediction


In [ ]:
# Import datasets
import os
import sparse
import json
import psycopg2
import numpy as np
import pandas as pd
import warnings
import itertools
import pyarrow as pa
import pyarrow.parquet as pq
import sqlalchemy as db
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, StratifiedKFold
from sklearn import metrics
import matplotlib.pyplot as plt
import scipy.stats
import seaborn as sns
from dask import dataframe as dd
from IPython.display import display, HTML 

In [ ]:
directory_features = '../databases/experiment_data/features/mortality_48h'

# Generate relevant static features
s = sparse.load_npz(f'{directory_features}/s.npz')

# Generate relevant temporal features
X = sparse.load_npz(f'{directory_features}/X.npz')

In [ ]:
# Display feature datasets


##### Load FIDDLE's mortality_48h task features into pandas
# Load static variables into a pandas table
s_names = json.load(open(f'{directory_features}/s.feature_names.json', 'r'))
df_S = pd.DataFrame(s.todense(), columns=s_names)

# Load temporal data into pandas table
X_names = json.load(open(f'{directory_features}/X.feature_names.json', 'r'))
df_X = pd.DataFrame(X.todense().reshape(-1, X.shape[-1]), columns=X_names)

display(df_S)
display(df_X)
list(df_S.columns.values)


In [ ]:
# Convert D_ITEMS into a usable dictionary to identify features and tables

D_items = open('../databases/mimic-iii-clinical-database-1.4-original/D_ITEMS.csv', 'r')
D_labitems = open('../databases/mimic-iii-clinical-database-1.4-original/D_LABITEMS.csv', 'r')
ID_dict = {}

name_code = {}

for line in D_items.readlines()[1:]:
    split = line.split(',')
    code = split[1]
    name = split[2].replace('"', '')
    table = split[5].replace('"', '') ### CHANGED
    
    ID_dict[code] = name
    name_code[name] = code

for line in D_labitems.readlines()[1:]:
    split = line.split(',')
    code = split[1]
    name = split[2].replace('"', '')
    table = split[5].replace('"', '') ### CHANGED
    
    ID_dict[code] = name
    name_code[name] = code


D_items.close()
D_labitems.close()

# print(ID_dict)

In [ ]:
def FIDDLE_features(feature_names: list) -> set:
    '''
    Extracts the feature names from the FIDDLE data.
    Args:
        feature_names: a list of column names from FIDDLE
    Returns:
        filtered_names: a set of feature names without other ID's and ranges from ordinal encoding
    '''
    filtered_names = []
    
    for i in feature_names:
        ind = i.find('_')
        code_name = i[:ind]
        
        # If format is 220048: ...
        if ' ' in code_name:
            continue
        
        # Convert to name using D_ITEMS if it's a code
        elif code_name.isnumeric():
            try:
                feature = ID_dict[code_name]
            
            # Code not found in D_ITEMS - TO FIX
            except:
                feature = code_name
        
        # Must be name like SysBP otherwise
        else:
            feature = code_name
        
        filtered_names.append(feature)
    
    unique_features = set(filtered_names)
    
    return unique_features


def mimic_features(path: str) -> list:
    '''
    Extracts the feature names retrieved from Yang et al. 2023 paper and returns a tuple that divides it based on
    temporal or static status.
    Args:
        path: path to the file of features
    Returns:
        features: a list containing two lists for temporal and static features
    '''
    features = [[], []]
    with open(path, 'r') as f:
        lines = f.readlines()
        for feature in range(len(lines)):
            # Before line 76 and line 76 is temporal
            if feature < 83:
                features[0].append(lines[feature][:-1])
            
            # After line 76 is static
            else:
                if lines[feature][-1] == '\n':
                    features[1].append(lines[feature][:-1])
                else:
                    features[1].append(lines[feature])
    features[1] = features[1][:-1]
    features[0].append('Measurement Time')

    return features

def similar_features(list1: set, list2: set) -> tuple:
    '''
    Discovers the elements that are shared in both list1 and list2.
    Args:
        list1: a list of elements
        list2: a list of elements
    Returns:
        num, shared: a tuple containing number of shared elements, and a list of the shared elements
    '''
    shared = []
    
    # Strip caps and whitespaces for less chance of missing shared features
    mod_list2 = []
    for i in list2:
        mod_list2.append(i.strip().lower())
    
    # Find shared elements
    for i in list1:
        if i.strip().lower() in mod_list2:
            shared.append(i)
    
    num = len(shared)
    return num, shared


##### Obtain unique feature values used in FIDDLE package

# Temporal features
print('TEMPORAL FEATURES FROM MORTALITY_48H FIDDLE EXPERIMENT')
print('-----------------------------------------------------------------------------------------------------------')
X_features = FIDDLE_features(X_names)
print(f'Number of temporal features: {len(X_features)}\n')
print(f'Features used: {X_features}\n')

# Static features
print('STATIC FEATURES FROM MORTALITY_48H FIDDLE EXPERIMENT')
print('-----------------------------------------------------------------------------------------------------------')

# REMOVED S_FEATURES FOR PRIVACY PURPOSES
print(f'Number of static features: {len(s_features)}\n')
print(f'Features used: {s_features}\n')


##### Obtain unique feature values used in Yoon et al. 2023 paper

path_features = '../databases/mimic-features.txt'
temp_features, static_features = mimic_features(path_features)

# Temporal features
print('\nTEMPORAL FEATURES FROM YANG ET AL. 2023 EXPERIMENT')
print('-----------------------------------------------------------------------------------------------------------')
print(f'Number of temporal features: {len(temp_features)}\n')
print(f'Features used: {temp_features}\n')

# Static features
print('STATIC FEATURES FROM YANG ET AL. 2023 EXPERIMENT')
print('-----------------------------------------------------------------------------------------------------------')
print(f'Number of static features: {len(static_features)}\n')
print(f'Features used: {static_features}\n')


##### Compare features present in both

# Temporal features
print('\nSHARED TEMPORAL FEATURES')
print('-----------------------------------------------------------------------------------------------------------')
numX, shared_X_features = similar_features(temp_features, X_features)
print(f'Number of shared temporal features: {numX}\n')
print(f'Temporal features shared:{shared_X_features}')


# Static features
print('\nSHARED STATUIC FEATURES')
print('-----------------------------------------------------------------------------------------------------------')
nums, shared_s_features = similar_features(static_features, s_features)
# REMOVED .APPEND METHOD FOR PRIVACY PURPOSES
nums += 1
print(f'Number of shared temporal features: {nums}\n')
print(f'Temporal features shared:{shared_s_features}')

## Importing data
### Assumptions
The following ID's and features have been assigned together due to their absences on the D_ITEMS.csv:

[REMOVED FOR PRIVACY PURPOSES]

Factors that were considered in choosing these alternatives include whether the label had an abbreviation the database source. Metavision was prioritised over Carevue in the decision.

In [ ]:
codes = []
c = 0
replacements = ['220050', '220051', '226512', '220045']

for i in X_features:
    try:
        codes.append(name_code[i])
    except:
        codes.append(replacements[c])
        c += 1

# changing each to int, so when it's sorted it goes in increasing order
for i in range(len(codes)):
    codes[i] = int(codes[i])
    
sorted_codes = sorted(codes)

# changing back to str so it can be used as a key value for code_name record
for i in range(len(sorted_codes)):
    sorted_codes[i] = str(sorted_codes[i])

# print sorted codes to each schema
fnl_str = ''
for i in sorted_codes:
    fnl_str += f"('{i}', pa.float32()),"

# Exploratory Data Analysis (EDA)

## Outcome definition of mortality
In-hospital mortality refers to a patient's death occurring within 28 days of admission, according to various studies (Churpek et al., 2021; Docherty et al., 2021; Group, 2021; UKHSA, 2022). './population/mortality_48h.csv' contains the 'mortality_LABEL' boolean field that represents whether the patient died at time T, where T >= 48 hours, in this case.

By observation, and concerning MIMIC-III's ADMISSIONS table, calculating the difference between ADMITTIME and DEATHTIME determines mortality. If the difference in time is less than T=48, then the corresponding example is excluded from the preprocessed data (Tang et al., 2020, p.22). If the patient has died, DISCHTIME should hold the same value as DEATHTIME.

The features will be aligned with the key hadm_id 'hospital admission id' from the admission table rather than subject_id from the patients table during EDA. Choosing the former option allows us to draw relationships between a patient's stay and their death, rather than the patient's attributes measured over potentially a course of many visits and their death. Selecting the latter may obscure the results.
## Features importance and frequency
The following are the top 10 features with highest importance from MIMIC-III that affects Google's mortality prediction model:
<ol>
<li>Tidal Volume (set)</li>
<li>Vti High</li>
<li>Tidal Volume (Obser)</li>
<li>Tidal Volume (observed)</li>
<li>Temperature F</li>
<li>calprevflg</li>
<li>SpO2 Alarm [High]</li>
<li>Temperature C (calc)</li>
<li>Resp Rate (Spont)</li>
<li>Tidal Volume (Set) (Yoon et al. supplementary information, 2023, p. 23)</li>
</ol>

The features found to have the greatest frequency in FIDDLE's preprocessed data, however, include:
<ul>
<li>heart rate</li>
<li>respiratory rate</li>
<li>temperature</li>
<li>systolic
blood pressure</li>
<li>diastolic blood pressure</li>
<li>peripheral oxygen
saturation (Tang et al., 2020, p.25)</li>
</ul>

## Table of Static Features in FIDDLE's experiment

In [ ]:
# Create a sqlalchemy engine to connect to database - ensure postgresql server is on
sqluser = 'postgres'
sqlpass = 'postgres'
dbname = 'mimic'
schema_name = 'mimiciii'
host = 'localhost'
port = 5432

engine = db.create_engine(f"postgresql+psycopg2://{sqluser}:{sqlpass}@{host}:{port}/{dbname}")
conn = engine.connect().execution_options(stream_results=True)

# removed features names for privacy purposes
query = f"""

SELECT

    -- Static features in FIDDLE's experiment. Remaining X features could not be located in MIMIC-III.
    
    [REMOVED FOR PRIVACY PURPOSES]

FROM {schema_name}.patients patients

LEFT JOIN {schema_name}.admissions adm
ON patients.subject_id = adm.subject_id

LEFT JOIN {schema_name}.icustays icu
ON patients.subject_id = icu.subject_id

"""

total_dfs = []

# process query via chunking
for chunk_dataframe in pd.read_sql(query, conn, chunksize=10000):
    total_dfs.append(chunk_dataframe)

# Concatenate all chunked dataframes then remove repeating icustays and itemid's
df = pd.concat(total_dfs)
display(df)


# Split ICU Stays Based on Patients

The data splits are stratified by patients. The patient ID will be linked to the icu stay IDs. Based on which dataset the patient belongs, the icu stay ID will be grouped into the same one. Then outcomes of each adversity type will be categorised accordingly. Note that the mortality partition split has already been completed. Currently found issues cross-referencing IDs, so has not been implemented for the final results.

In [ ]:
edited = False

def count_raw(ls, element):
    '''
    Remove all instances of an element from an array.
    '''
    new_ls = [i for i in ls if i != element] 
    return new_ls

# Only needs execution one time locally
if not edited:
    split_dir = '../databases/experiment_data/data_split'

    # Define train, test, val for patient IDs
    train_arr = []
    test_arr = []
    val_arr = []

    train_patient = pd.read_csv(f'{split_dir}/train_listfile.csv')['stay']
    for i in train_patient.index:

        # Filter each row to only produce patient ID
        row = train_patient.iloc[i]
        train_arr.append(row[:row.index('_')])

    test_patient = pd.read_csv(f'{split_dir}/test_listfile.csv')['stay']
    for i in test_patient.index:

        row = test_patient.iloc[i]
        test_arr.append(row[:row.index('_')])

    val_patient = pd.read_csv(f'{split_dir}/val_listfile.csv')['stay']
    for i in val_patient.index:

        row = val_patient.iloc[i]
        val_arr.append(row[:row.index('_')])
    
    # Create a dictionary for patient ID:ICU ID
    patient_to_icu = pd.read_csv(f'{split_dir}/all_stays.csv')
    patient_to_icu = patient_to_icu[['SUBJECT_ID', 'ICUSTAY_ID']]    
    
    dict_p_icu = patient_to_icu.set_index('ICUSTAY_ID')['SUBJECT_ID'].to_dict()
    
    # Manually label each outcome for each adversity type
    # ARF 4h
    arf_4h_table = pd.read_csv(f'{split_dir}/ARF_4h.csv')
    partition_data = []
    icu_train = []
    icu_val = []
    icu_test = []
    
    unclassified_valid_id = []
    invalid_id = []
    # Assign partition label based on patient ID
    for i in arf_4h_table.index:
        try:
            icu_id = arf_4h_table['ID'].iloc[i]
            patient_id = str(dict_p_icu[icu_id])
            
        
            if patient_id in train_arr:
                partition_data.append('train')

            elif patient_id in test_arr:
                partition_data.append('test')

            elif patient_id in val_arr:
                partition_data.append('val')

            # Patient ID is valid and not classified in train, test, or val
            else:
                unclassified_valid_id.append(patient_id)
                
                # To keep corresponding examples across files consistent
                partition_data.append('X')
        
        # ICU ID in data files do not have a corresponding patient ID
        except:
            invalid_id.append(icu_id)
            partition_data.append('X')
            
#     print(partition_data) 
        
        
# Read patient id and correspond it with icu id
        

In [ ]:
print(f'Total number of examples: {len(partition_data)}')
print(f'Total number of classified examples: {len(count_raw(partition_data, "X"))}')
print(f'Number valid ICU IDs in which their corresponding patient ID is not classified according to FIDDLEs source code files: {len(unclassified_valid_id)}')
print(f'Number of invalid ICU IDs found in physionets dataset: {len(invalid_id)}')

# Replication of FIDDLE's mortality experiment

## Loading Datasets

In [ ]:
y_directory = '../databases/experiment_data/population/mortality_48h.csv'
directory_features = '../databases/experiment_data/features/mortality_48h'

# Load independent variables
s = sparse.load_npz(f'{directory_features}/s.npz').todense()
X = sparse.load_npz(f'{directory_features}/X.npz').todense()

# Load dependent variable - mortality outcome
y = pd.read_csv(y_directory)['y_true']

N,L,D = X.shape
_,d = s.shape

# Combine information to create full datasets - assuming order of y-labels matches order of independent features
X_all = np.hstack([s, X.reshape((N,L*D))])
y_all = y[:N]

scaler = MinMaxScaler()
X_all = scaler.fit_transform(X_all)


## Combining Training and Validation Datasets for Training

FIDDLE's experiment combines the training and validation test sets for the logistic regression model on adversity outcomes (Page 7, Tang et al. 2021, Supplementary Information). A 5-fold cross validation method is used to evaluate the performance of the model with CV and a random search budget of 50.

In [ ]:
# Creating the 1) training + validation set, and 2) testing set separately.
X_train_arr = []
y_train_arr = []
 
X_test_arr = []
y_test_arr = []

split_metadata = pd.read_csv(y_directory)['partition']

for partition in range(len(split_metadata)):
    
    part = split_metadata[partition]
    
    # Classify cross-validation set
    if part == 'train' or part == 'val':
        
        # Append corresponding example + outcome to training set
        X_train_arr.append(X_all[partition])
        y_train_arr.append(y_all[partition])
        
    # Classify testing set
    elif part == 'test':
        
        # Append corresponding example + outcome to testing set
        X_test_arr.append(X_all[partition])
        y_test_arr.append(y_all[partition])
        
    else:
        print('uh oh D:')

X_train = np.array(X_train_arr)
y_train = np.array(y_train_arr)

X_test = np.array(X_test_arr)
y_test = np.array(y_test_arr)


## Training LR Model

In [ ]:
"""Following specs were used to train:
- mortality_model = LogisticRegression(solver='lbfgs', max_iter=10000)
- macbook (16gb RAM, 2.3 GHz 8-Core Intel Core i9)
- data split matches experiment

"""
cross_val = False

if cross_val:

    search_budget = 50
    n_jobs = 50

    mortality_model = RandomizedSearchCV(
        LogisticRegression(solver='lbfgs'), 
        {'C': scipy.stats.reciprocal(1e-5, 1e5)},
        n_iter=search_budget,
        cv=StratifiedKFold(5),
        scoring='roc_auc',
        n_jobs=n_jobs, 
        verbose=2
    )

elif not cross_val:
    mortality_model = LogisticRegression(solver='lbfgs', max_iter=10000, C=0.005)

In [ ]:
# Train and store the model if file doesn't exist. Otherwise model reads from file.
path_model = 'trained_models_2/mortality_48h.sav'

if os.path.exists(path_model):
    # Read pickle file
    mortality_model = pickle.load(open(path_model, 'rb'))
    
else:
    # Train model and store in file
    mortality_model.fit(X_train, y_train)
    pickle.dump(mortality_model, open(path_model, 'wb'))

# Replication of FIDDLE's ARF 4h Experiment

## Loading Datasets

In [ ]:
y_directory = '../databases/experiment_data/population/ARF_4h.csv'
directory_features = '../databases/experiment_data/features/ARF_4h'

# Load independent variables
s = sparse.load_npz(f'{directory_features}/s.npz').todense()
X = sparse.load_npz(f'{directory_features}/X.npz').todense()

# Load dependent variable
y = pd.read_csv(y_directory)['ARF_LABEL']

N,L,D = X.shape
_,d = s.shape

X_all = np.hstack([s, X.reshape((N,L*D))])
y_all = y[:N]

scaler = MinMaxScaler()
X_all = scaler.fit_transform(X_all)


## Creating Training and Testing Datasets for Training

Partitions for ARF and shock outcomes are unlabelled and were not published by FIDDLE. Random state 0 will be used to create datasets with a train:test ratio of 85:15.

In [ ]:
# FIND OUT MORE ABOUT PARTITIONS SINCE A COLUMN IS NOT GIVEN
# Manually creating a training and testing split.

X_train_arf4, X_test_arf4, y_train_arf4, y_test_arf4 = train_test_split(X_all, y_all, test_size=0.15, random_state=0)

## Training LR Model

In [ ]:
"""Following specs were used to train model:
- same hyperparameters as FIDDLE except n_jobs = 7
- macbook (16gb RAM, 2.3 GHz 8-Core Intel Core i9)
- data split does not match experiment
"""


cross_val = True

if cross_val:

    search_budget = 50
    n_jobs = 50

    arf_4h_model = RandomizedSearchCV(
        LogisticRegression(solver='lbfgs', max_iter=100000), 
        {'C': scipy.stats.reciprocal(1e-5, 1e5)},
        n_iter=search_budget,
        cv=StratifiedKFold(5),
        scoring='roc_auc',
        n_jobs=n_jobs,
        verbose=2
    )

elif not cross_val:
    arf_4h_model = LogisticRegression(solver='lbfgs', max_iter=100000)

In [ ]:
# Train and store the model if file doesn't exist. Otherwise model reads from file.
path_model = 'trained_models_2/arf_4h.sav'

if os.path.exists(path_model):
    # Read pickle file
    arf_4h_model = pickle.load(open(path_model, 'rb'))
    
else:
    # Train model and store in file
    arf_4h_model.fit(X_train_arf4, y_train_arf4)
    pickle.dump(arf_4h_model, open(path_model, 'wb'))

# Replication of FIDDLE's ARF 12h Experiment

## Loading Datasets

In [ ]:
y_directory = '../databases/experiment_data/population/ARF_12h.csv'
directory_features = '../databases/experiment_data/features/ARF_12h'

# Load independent variables
s = sparse.load_npz(f'{directory_features}/s.npz').todense()
X = sparse.load_npz(f'{directory_features}/X.npz').todense()

# Load dependent variable
y = pd.read_csv(y_directory)['ARF_LABEL']

N,L,D = X.shape
_,d = s.shape

X_all = np.hstack([s, X.reshape((N,L*D))])
y_all = y[:N]

scaler = MinMaxScaler()
X_all = scaler.fit_transform(X_all)


## Creating Training and Testing Datasets for Training

In [ ]:
X_train_arf12, X_test_arf12, y_train_arf12, y_test_arf12 = train_test_split(X_all, y_all, test_size=0.15, random_state=0)

## Training LR Model

In [ ]:
"""Following specs were used to train model:
- same hyperparameters as FIDDLE except n_jobs = 3
- desktop (32gb RAM, AMD Ryzen 3 3300X 4-Core Processor)
- data split does not match experiment
"""
cross_val = False

if cross_val:

    search_budget = 50
    n_jobs = 50

    arf_12h_model = RandomizedSearchCV(
        LogisticRegression(solver='lbfgs'), 
        {'C': scipy.stats.reciprocal(1e-5, 1e5)},
        n_iter=search_budget,
        cv=StratifiedKFold(5),
        scoring='roc_auc',
        n_jobs=n_jobs, 
        verbose=2
    )

elif not cross_val:
    arf_12h_model = LogisticRegression(solver='lbfgs', max_iter=100000)

In [ ]:
# Train and store the model if file doesn't exist. Otherwise model reads from file.
path_model = 'trained_models_2/arf_12h.sav'

if os.path.exists(path_model):
    # Read pickle file
    arf_12h_model = pickle.load(open(path_model, 'rb'))
    
else:
    # Train model and store in file
    arf_12h_model.fit(X_train_arf12, y_train_arf12)
    pickle.dump(arf_12h_model, open(path_model, 'wb'))

# Replication of FIDDLE's Shock 4h Experiment

## Loading Datasets

In [ ]:
y_directory = '../databases/experiment_data/population/Shock_4h.csv'
directory_features = '../databases/experiment_data/features/Shock_4h'

# Load independent variables
s = sparse.load_npz(f'{directory_features}/s.npz').todense()
X = sparse.load_npz(f'{directory_features}/X.npz').todense()

# Load dependent variable
y = pd.read_csv(y_directory)['Shock_LABEL']

N,L,D = X.shape
_,d = s.shape

X_all = np.hstack([s, X.reshape((N,L*D))])
y_all = y[:N]

scaler = MinMaxScaler()
X_all = scaler.fit_transform(X_all)


## Creating Training and Testing Datasets for Training

In [ ]:
X_train_shock4, X_test_shock4, y_train_shock4, y_test_shock4 = train_test_split(X_all, y_all, test_size=0.15, random_state=0)

## Training LR Model

In [ ]:
"""Following specs were used to train model:
- same hyperparameters as FIDDLE except n_jobs = 3
- desktop (32gb RAM, AMD Ryzen 3 3300X 4-Corde Processor)
- data split does not match experiment
"""

# Change to true if you have enough RAM to compute 250 fits
cross_val = False

if cross_val:

    search_budget = 50
    n_jobs = 50

    shock_4h_model = RandomizedSearchCV(
        LogisticRegression(solver='lbfgs'), 
        {'C': scipy.stats.reciprocal(1e-5, 1e5)},
        n_iter=search_budget,
        cv=StratifiedKFold(5),
        scoring='roc_auc',
        n_jobs=n_jobs, 
        verbose=2
    )

elif not cross_val:
    shock_4h_model = LogisticRegression(solver='lbfgs', max_iter=100000)

In [ ]:
# Train and store the model if file doesn't exist. Otherwise model reads from file.
path_model = 'trained_models_2/shock_4h.sav'

if os.path.exists(path_model):
    # Read pickle file
    shock_4h_model = pickle.load(open(path_model, 'rb'))
    
else:
    # Train model and store in file
    shock_4h_model.fit(X_train_shock4, y_train_shock4)
    pickle.dump(shock_4h_model, open(path_model, 'wb'))

# Replication of FIDDLE's Shock 12h Experiment

## Loading Datasets

In [ ]:
y_directory = '../databases/experiment_data/population/Shock_12h.csv'
directory_features = '../databases/experiment_data/features/Shock_12h'

# Load independent variables
s = sparse.load_npz(f'{directory_features}/s.npz').todense()
X = sparse.load_npz(f'{directory_features}/X.npz').todense()

# Load dependent variable
y = pd.read_csv(y_directory)['Shock_LABEL']

N,L,D = X.shape
_,d = s.shape

X_all = np.hstack([s, X.reshape((N,L*D))])
y_all = y[:N]

scaler = MinMaxScaler()
X_all = scaler.fit_transform(X_all)


## Creating Training and Testing Datasets for Training

In [ ]:
X_train_shock12, X_test_shock12, y_train_shock12, y_test_shock12 = train_test_split(X_all, y_all, test_size=0.15, random_state=0)

## Training LR Model

In [ ]:
"""Following specs were used to train model:
- same hyperparameters as FIDDLE except n_jobs = 3
- desktop (32gb RAM, AMD Ryzen 3 3300X 4-Corde Processor)
- data split does not match experiment
"""
# Change to true if you have enough RAM to compute 250 fits
cross_val = False

if cross_val:

    search_budget = 50
    n_jobs = 50

    shock_12h_model = RandomizedSearchCV(
        LogisticRegression(solver='lbfgs'), 
        {'C': scipy.stats.reciprocal(1e-5, 1e5)},
        n_iter=search_budget,
        cv=StratifiedKFold(5),
        scoring='roc_auc',
        n_jobs=n_jobs, 
        verbose=2
    )

elif not cross_val:
    shock_12h_model = LogisticRegression(solver='lbfgs', max_iter=100000)

In [ ]:
# Train and store the model if file doesn't exist. Otherwise model reads from file.
path_model = 'trained_models_2/shock_12h.sav'

if os.path.exists(path_model):
    # Read pickle file
    shock_12h_model = pickle.load(open(path_model, 'rb'))
    
else:
    # Train model and store in file
    shock_12h_model.fit(X_train_shock12, y_train_shock12)
    pickle.dump(shock_12h_model, open(path_model, 'wb'))

# Evaluating the Models

## Calculating AUROC and AUPR Scores
Note that AUPRC has been calculated by directly calculating the area under the precision-recall curve using the trapezoidal rule instead of using the average_precision_score() method to increase accuracy of results.

In [ ]:
cross_val = False

if cross_val:
    pass
elif not cross_val:
    # In-hospital mortality 48h outcome
    prediction_test_scores_mortality = mortality_model.decision_function(X_test)
    mortality_predict = mortality_model.predict(X_test)
    
    auroc_mortality = metrics.roc_auc_score(y_test, prediction_test_scores_mortality)
    # Returns arrays containing precision, recall, thresholds. Then thresholds are removed, array positions are 
    # reversed. Area under that curve is calculated using trapezoidal rule.
    aupr_mortality = metrics.auc(*metrics.precision_recall_curve(y_test, mortality_predict)[1::-1])

    # ARF 4h
    scores_arf_4h = arf_4h_model.decision_function(X_test_arf4)
    arf_4h_predict = arf_4h_model.predict(X_test_arf4)

    auroc_arf_4h = metrics.roc_auc_score(y_test_arf4, scores_arf_4h)
    aupr_arf_4h = metrics.auc(*metrics.precision_recall_curve(y_test_arf4, arf_4h_predict)[1::-1])

    # ARF 12h
    scores_arf_12h = arf_12h_model.decision_function(X_test_arf12)
    arf_12h_predict = arf_12h_model.predict(X_test_arf12)
    
    auroc_arf_12h = metrics.roc_auc_score(y_test_arf12, scores_arf_12h)
    aupr_arf_12h = metrics.auc(*metrics.precision_recall_curve(y_test_arf12, arf_12h_predict)[1::-1])
    
    
    # Shock 4h
    scores_shock_4h = shock_4h_model.decision_function(X_test_shock4)
    shock_4h_predict = shock_4h_model.predict(X_test_shock4)

    auroc_shock_4h = metrics.roc_auc_score(y_test_shock4, scores_shock_4h)
    aupr_shock_4h = metrics.auc(*metrics.precision_recall_curve(y_test_shock4, shock_4h_predict)[1::-1])

    
    # Shock 12h
    scores_shock_12h = shock_12h_model.decision_function(X_test_shock12)
    shock_12h_predict = shock_12h_model.predict(X_test_shock12)

    auroc_shock_12h = metrics.roc_auc_score(y_test_shock12, scores_shock_12h)
    aupr_shock_12h = metrics.auc(*metrics.precision_recall_curve(y_test_shock12, shock_12h_predict)[1::-1])

    

In [ ]:
data = [[0.856,0.444,0.817,0.657,0.757,0.291,0.825,0.548,0.792,0.274],
        [
            auroc_mortality,aupr_mortality,
            auroc_arf_4h,aupr_arf_4h,
            auroc_arf_12h,aupr_arf_12h,
            auroc_shock_4h,aupr_shock_4h,
            auroc_shock_12h,aupr_shock_12h
        ], 
        [0,0,0,0,0,0,0,0,0,0]
       ]

header = pd.MultiIndex.from_product([['In-hospital mortality 48h',
                                      'Acute Respiratory Failure 4h', 
                                      'Acute Respiratory Failure 12h',
                                      'Shock 4h',
                                      'Shock 12h'],
                                     ['AUROC', 'AUPR']],
                                    names=['Adversity Outcome','Metric'])
metrics_table = pd.DataFrame(data, 
                  index=['Original','Replicated','Replicated Synthetic'], 
                  columns=header)
metrics_table = metrics_table.round(3)
display(metrics_table)

## Hyperparameters used

In [ ]:
# Display hyperparameters used for each experiment

mortality_C = mortality_model.get_params()['C'] # Randomised Search CV was not used
arf_4h_C = arf_4h_model.best_params_['C']
arf_12h_C = arf_12h_model.best_params_['C']
shock_4h_C = shock_4h_model.best_params_['C']
shock_12h_C = shock_12h_model.best_params_['C']

hyperparameters = ["Inverse of Regularisation Strength ('C')"]

hyper_table = pd.DataFrame(
    {
        'In-hospital mortality 48h': [mortality_C],
        'Acute Respiratory Failure 4h': [arf_4h_C],
        'Acute Respiratory Failure 12h': [arf_12h_C],
        'Shock 4h': [shock_4h_C],
        'Shock 12h': [shock_12h_C]
    },
    index = hyperparameters

)

display(hyper_table)

## Confusion Matrices

Definition of labels:
<ul>
<li>0 indicates that the patient does not experience the adverse outcome after t=T hours of being admitted into ICU</li>
<li>1 indicates that the patient does experience the adverse outcome after t=T hours of being admitted into ICU.</li>
</ul>
General trends:
<ul>
    <li> There seems to be a high proportion of False Negative results (bottom-left)</li>
</ul>

### In-hospital mortality 48h

In [ ]:
# https://medium.com/mlearning-ai/heatmap-for-correlation-matrix-confusion-matrix-extra-tips-on-machine-learning-b0377cee31c2

sns.set(font_scale=1.4)
conf_mtrx = metrics.confusion_matrix(y_test, mortality_predict)
test_counts = ["{0:0.0f}".format(value) for value in conf_mtrx.flatten()]
test_percentage = ["{0:.2%}".format(value) for value in conf_mtrx .flatten()/np.sum(conf_mtrx)]
test_labels = [f"{v2}\n{v3}" for v2, v3 in zip(test_counts,test_percentage)]
test_labels = np.asarray(test_labels).reshape(2,2)

plt.figure(figsize = (12,5))
display_conf_mtrx = sns.heatmap(conf_mtrx, annot=test_labels, fmt='', cmap='Blues')
display_conf_mtrx.set_xlabel('Predicted Label', fontsize=25)
display_conf_mtrx.set_ylabel('Actual Label', fontsize=25)
plt.show()

### Acute Respiratory Failure 4h

In [ ]:
sns.set(font_scale=1.4)
conf_mtrx = metrics.confusion_matrix(y_test_arf4, arf_4h_predict)
test_counts = ["{0:0.0f}".format(value) for value in conf_mtrx.flatten()]
test_percentage = ["{0:.2%}".format(value) for value in conf_mtrx .flatten()/np.sum(conf_mtrx)]
test_labels = [f"{v2}\n{v3}" for v2, v3 in zip(test_counts,test_percentage)]
test_labels = np.asarray(test_labels).reshape(2,2)

plt.figure(figsize = (12,5))
display_conf_mtrx = sns.heatmap(conf_mtrx, annot=test_labels, fmt='', cmap='Blues')
display_conf_mtrx.set_xlabel('Predicted Label', fontsize=25)
display_conf_mtrx.set_ylabel('Actual Label', fontsize=25)
plt.show()

### Acute Respiratory Failure 12h

In [ ]:
sns.set(font_scale=1.4)
conf_mtrx = metrics.confusion_matrix(y_test_arf12, arf_12h_predict)
test_counts = ["{0:0.0f}".format(value) for value in conf_mtrx.flatten()]
test_percentage = ["{0:.2%}".format(value) for value in conf_mtrx .flatten()/np.sum(conf_mtrx)]
test_labels = [f"{v2}\n{v3}" for v2, v3 in zip(test_counts,test_percentage)]
test_labels = np.asarray(test_labels).reshape(2,2)

plt.figure(figsize = (12,5))
display_conf_mtrx = sns.heatmap(conf_mtrx, annot=test_labels, fmt='', cmap='Blues')
display_conf_mtrx.set_xlabel('Predicted Label', fontsize=25)
display_conf_mtrx.set_ylabel('Actual Label', fontsize=25)
plt.show()

### Shock 4h

In [ ]:
sns.set(font_scale=1.4)
conf_mtrx = metrics.confusion_matrix(y_test_shock4, shock_4h_predict)
test_counts = ["{0:0.0f}".format(value) for value in conf_mtrx.flatten()]
test_percentage = ["{0:.2%}".format(value) for value in conf_mtrx .flatten()/np.sum(conf_mtrx)]
test_labels = [f"{v2}\n{v3}" for v2, v3 in zip(test_counts,test_percentage)]
test_labels = np.asarray(test_labels).reshape(2,2)

plt.figure(figsize = (12,5))
display_conf_mtrx = sns.heatmap(conf_mtrx, annot=test_labels, fmt='', cmap='Blues')
display_conf_mtrx.set_xlabel('Predicted Label', fontsize=25)
display_conf_mtrx.set_ylabel('Actual Label', fontsize=25)
plt.show()

### Shock 12h

In [ ]:
sns.set(font_scale=1.4)
conf_mtrx = metrics.confusion_matrix(y_test_shock12, shock_12h_predict)
test_counts = ["{0:0.0f}".format(value) for value in conf_mtrx.flatten()]
test_percentage = ["{0:.2%}".format(value) for value in conf_mtrx .flatten()/np.sum(conf_mtrx)]
test_labels = [f"{v2}\n{v3}" for v2, v3 in zip(test_counts,test_percentage)]
test_labels = np.asarray(test_labels).reshape(2,2)

plt.figure(figsize = (12,5))
display_conf_mtrx = sns.heatmap(conf_mtrx, annot=test_labels, fmt='', cmap='Blues')
display_conf_mtrx.set_xlabel('Predicted Label', fontsize=25)
display_conf_mtrx.set_ylabel('Actual Label', fontsize=25)
plt.show()

## ROC Curves and PR Curves

Each PR Curve has a different baseline, while ROC curves can be compared to the same naive classifer metric. 

### In-hospital Mortality 48h

In [ ]:
# Output of predict_proba method is [% of 0 outcome, % of 1 outcome]
prediction_prob_mortality48h = mortality_model.predict_proba(X_test)

# [:, 1] selects all rows, then selects column with index 1. Purpose is to get probabilities of all positive outcomes.
fpr_mortality48h, tpr_mortality48h, thresh_mortality48h = metrics.roc_curve(y_test, prediction_prob_mortality48h[:,1], pos_label=1)

mortality_precision, mortality_recall, _ = metrics.precision_recall_curve(y_test, prediction_prob_mortality48h[:,1])

# true positive rate = false positive rate
random_probs = [0 for i in range(len(y_test))]
x_mortality48h, y_mortality48h, _ = metrics.roc_curve(y_test, random_probs, pos_label=1)

### ARF 4h

In [ ]:
prediction_prob_arf4h = arf_4h_model.predict_proba(X_test_arf4)

# ROC
fpr_arf4h, tpr_arf4h, thresh_arf4h = metrics.roc_curve(y_test_arf4, prediction_prob_arf4h[:,1], pos_label=1)

# PR
arf4h_precision, arf4h_recall, _ = metrics.precision_recall_curve(y_test_arf4, prediction_prob_arf4h[:,1])

### ARF 12h

In [ ]:
prediction_prob_arf12h = arf_12h_model.predict_proba(X_test_arf12)

# ROC
fpr_arf12h, tpr_arf12h, thresh_arf12h = metrics.roc_curve(y_test_arf12, prediction_prob_arf12h[:,1], pos_label=1)

# PR
arf12h_precision, arf12h_recall, _ = metrics.precision_recall_curve(y_test_arf12, prediction_prob_arf12h[:,1])

### Shock 4h

In [ ]:
prediction_prob_shock4h = shock_4h_model.predict_proba(X_test_shock4)

# ROC
fpr_shock4h, tpr_shock4h, thresh_shock4h = metrics.roc_curve(y_test_shock4, prediction_prob_shock4h[:,1], pos_label=1)

# PR
shock4h_precision, shock4h_recall, _ = metrics.precision_recall_curve(y_test_shock4, prediction_prob_shock4h[:,1])

### Shock 12h

In [ ]:
prediction_prob_shock12h = shock_12h_model.predict_proba(X_test_shock12)

# ROC
fpr_shock12h, tpr_shock12h, thresh_shock12h = metrics.roc_curve(y_test_shock12, prediction_prob_shock12h[:,1], pos_label=1)

# PR
shock12h_precision, shock12h_recall, _ = metrics.precision_recall_curve(y_test_shock12, prediction_prob_shock12h[:,1])

In [ ]:
# plot roc curves
plt.plot(fpr_mortality48h, tpr_mortality48h, linestyle='--', color='red', label="Mortality 48h")
plt.plot(fpr_arf4h, tpr_arf4h, linestyle='--',color='orange', label='ARF 4h')
plt.plot(fpr_arf12h, tpr_arf12h, linestyle='--',color='yellow', label='ARF 12h')
plt.plot(fpr_shock4h, tpr_shock4h, linestyle='--',color='green', label='Shock 4h')
plt.plot(fpr_shock12h, tpr_shock12h, linestyle='--',color='blue', label='Shock 12h')

plt.plot(x_mortality48h, y_mortality48h, linestyle='--', color='purple')

# title and labels
plt.title('ROC Curves for FIDDLE Experiments')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.legend(loc='best')
plt.show()

In [ ]:
# Plot PR Curves

# In-hospital mortality 48h
# Calculate the baseline by assuming all outcomes are predicted positive
baseline_mortality = len(y_test[y_test==1]) / len(y_test)
plt.plot([0, 1], [baseline_mortality, baseline_mortality], linestyle='--', label='Baseline')
plt.plot(mortality_recall, mortality_precision, linestyle='--', label='Mortality 48h')

# Title and labels
plt.title('Precision-Recall Curve for In-Hospital Mortality 48h')
plt.xlabel('Recall')
plt.ylabel('Precision')

plt.legend(loc='best')
plt.show()

In [ ]:
baseline_arf4h = len(y_test_arf4[y_test_arf4==1]) / len(y_test_arf4)

# ARF 4h
plt.plot([0, 1], [baseline_arf4h, baseline_arf4h], linestyle='--', label='Baseline')
plt.plot(arf4h_recall, arf4h_precision, linestyle='--', label='ARF 4h')

# Title and labels
plt.title('Precision-Recall Curve for ARF 4h')
plt.xlabel('Recall')
plt.ylabel('Precision')

plt.legend(loc='best')
plt.show()

In [ ]:
baseline_arf12h = len(y_test_arf12[y_test_arf12==1]) / len(y_test_arf12)

# ARF 12h
plt.plot([0, 1], [baseline_arf12h, baseline_arf12h], linestyle='--', label='Baseline')
plt.plot(arf12h_recall, arf12h_precision, linestyle='--', label='ARF 12h')

# Title and labels
plt.title('Precision-Recall Curve for ARF 12h')
plt.xlabel('Recall')
plt.ylabel('Precision')

plt.legend(loc='best')
plt.show()

In [ ]:
baseline_shock4h = len(y_test_shock4[y_test_shock4==1]) / len(y_test_shock4)

# Shock 4h
plt.plot([0, 1], [baseline_shock4h, baseline_shock4h], linestyle='--', label='Baseline')
plt.plot(shock4h_recall, shock4h_precision, linestyle='--', label='Shock 4h')

# Title and labels
plt.title('Precision-Recall Curve for Shock 4h')
plt.xlabel('Recall')
plt.ylabel('Precision')

plt.legend(loc='best')
plt.show()

In [ ]:
baseline_shock12h = len(y_test_shock12[y_test_shock12==1]) / len(y_test_shock12)

# Shock 12h
plt.plot([0, 1], [baseline_shock12h, baseline_shock12h], linestyle='--', label='Baseline')
plt.plot(shock12h_recall, shock12h_precision, linestyle='--', label='Shock 12h')

# Title and labels
plt.title('Precision-Recall Curve for Shock 12h')
plt.xlabel('Recall')
plt.ylabel('Precision')

plt.legend(loc='best')
plt.show()